In [ ]:
import sys
import os
os.environ['OPP4SAR_DIR'] = '/home/jhewers/Documents/meng_project/code/'
sys.path.insert(0,os.getenv('OPP4SAR_DIR'))
DIR = os.path.join(os.getenv('OPP4SAR_DIR'),'generated_data/prob_map_jackton')

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 300

import json
import numpy as np
import scipy.stats as st
import scipy as sp
import os
import csv
from PIL import ImageColor, Image

import MP as MercatorProjection
import utm

from src.json_helpers import GlobalJsonDecoder
from src.data_models.positional.waypoint import Waypoint, Waypoints
from src.data_models.probability_map import ProbabilityMap
from src.waypoint_generation import WaypointFactory, WaypointAlgSettings
from src.enums import WaypointAlgorithmEnum
from src.simulation.vehicle import VehicleSimData

In [ ]:
from src.waypoint_generation import CostFunc
cost_func = CostFunc()

wp_gen_settings = WaypointAlgSettings.Global()

In [ ]:
with open(os.path.join(DIR,"output_sar_ps.json"),'r') as f:
    data_sar = json.load(f,cls=GlobalJsonDecoder)
    print(type(data_sar))
with open(os.path.join(DIR,"output_sim_ps.json"), 'r') as f:
    data_sim = json.load(f,cls=GlobalJsonDecoder)
    print(type(data_sim))
with open(os.path.join(DIR,"output_wp_ps.json"), 'r') as f:
    data_wp = json.load(f,cls=GlobalJsonDecoder)
    print(type(data_wp))

R = 15 

In [ ]:
import AirDataUAV as ADU 
from PIL import Image

d1 = ADU.UAVData("/home/jhewers/Documents/meng_project/ps_data/2020-11-19_14-18-29.csv")
d2 = ADU.UAVData("/home/jhewers/Documents/meng_project/ps_data/2020-11-19_14-39-52.csv")
d3 = ADU.UAVData("/home/jhewers/Documents/meng_project/ps_data/2020-11-19_15-16-33.csv")

img = ProbabilityMap.fromPNG("/home/jhewers/Documents/meng_project/code/img/probability_imgs/prob_map_8_jackton.png")
img_location = ProbabilityMap.fromPNG("/home/jhewers/Documents/meng_project/code/img/probability_imgs/prob_map_8_jackton_map.png")

center = MercatorProjection.G_LatLng(55.751849, -4.238594)
corners = MercatorProjection.getCorners(center, 17, 1280/2,1280/2)
N,W,_,_ = utm.from_latlon(corners['N'],corners['W'])
S,E,_,_ = utm.from_latlon(corners['S'],corners['E'])

w_m,h_m = (W-E,S-N)
ORIGIN = np.array([N,W])

img = img.resampled(int(h_m),int(w_m))
img_location = img_location.resampled(int(h_m),int(w_m))

for d in [d1,d2,d3]:
    d.utm[0] -= ORIGIN[0]
    d.utm[1] = -(d.utm[1] - ORIGIN[1])

plt.imshow(img_location.toIMG())
plt.imshow(img.toIMG(),alpha = 0.4)

for dat in data_sim.data:
    plt.plot(dat[1].pos.x,[f-10 for f in dat[1].pos.y],label=str(dat[0]).split('.')[1])
    plt.scatter(dat[1].pos.x[0],[f-10 for f in dat[1].pos.y][0])
    plt.scatter(203, 111)

#plt.scatter([f.x for f in data_sar.data],[f.y for f in data_sar.data])

plt.plot(d1.utm[0],d1.utm[1],label="PS ASU Inspector")
plt.plot(d2.utm[0],d2.utm[1],label="PS AUS Steven Preece")
plt.plot(d3.utm[0],d3.utm[1],label='Attempted Parallel Swaths')
plt.legend()
plt.ylabel("y (m)")
plt.xlabel("x (m)")
fig=plt.gcf()
fig.savefig(os.path.join(DIR,'paths_ps.png'))
plt.show()


In [ ]:
npts = 25
for d,label in zip([d1,d2,d3],["PS ASU Inspector","PS AUS Steven Preece","Attempted Parallel Swaths"]):
    file_path = os.path.join(DIR,f"prob_accum_w_t_{'_'.join(label.split())}.csv")
    t_arr= []
    c_arr = []
    if os.path.isfile(file_path):
        print(f"File {file_path} found")
        with open(file_path,'r') as f:
            csvreader = csv.DictReader(f) 
            for row in csvreader:
                c_arr.append(float(row['cost']))
                t_arr.append(float(row['time']))
    else:
        print(f"File {file_path} NOT found")
        wp_arr = []
        x = d.utm[0]
        y = d.utm[1]
        print(f"{label} - {npts} points to calculate...")
        for i in range(0,len(x),len(x)//npts):
            xi = x[i]
            yi = y[i]
            ti = d.t_s[i]
            wp_arr.append(Waypoint(xi,yi))
            wps = Waypoints(wp_arr)
            cost = cost_func.calculate(wps,img,R)
            t_arr.append(ti)
            c_arr.append(cost)    
            print(f"{label}: {i}/{len(x)}")        
        with open(file_path,'w') as f:
            csvwriter = csv.writer(f)
            csvwriter.writerow(['time','cost'])
            csvwriter.writerows([(f,g) for f,g in zip(t_arr,c_arr)])
    
    plt.plot(t_arr,c_arr, label=label)
    print(f"Final cost for {label} is {c_arr[-1]:.4f}")

alg,vehicle = data_sim.data[0]
file_path = os.path.join(DIR,f"prob_accum_w_t_{str(alg).split('.')[1].lower()}.csv")
t_arr= []
c_arr = []
if os.path.isfile(file_path):
    print(f"File {file_path} found")
    with open(file_path,'r') as f:
        csvreader = csv.DictReader(f) 
        for row in csvreader:
            c_arr.append(float(row['cost']))
            t_arr.append(float(row['time']))
else:
    wp_arr = []
    x = np.array(vehicle.pos.x)
    y = np.array(vehicle.pos.y)
    t = vehicle.t
    print(f"{alg} - {npts} points to calculate...")
    for i in range(0,len(x),len(x)//npts) :
        xi = x[i]
        yi = y[i]
        ti = t[i]
        wp_arr.append(Waypoint(xi,yi))
        wps = Waypoints(wp_arr)
        cost = cost_func.calculate(wps,img,R)
        t_arr.append(ti)
        c_arr.append(cost)    
        print(f"{alg} data: {i}/{len(x)}")
    with open(file_path,'w') as f:
            csvwriter = csv.writer(f)
            csvwriter.writerow(['time','cost'])
            csvwriter.writerows([(f,g) for f,g in zip(t_arr,c_arr)])
print(f"Final cost for {alg} is {c_arr[-1]:.4f}")
plt.plot(t_arr,c_arr, label=str(alg).split('.')[1])

plt.xlabel("Time (s)")
plt.ylabel("Accumulated probability")
plt.ylim([0, -1])
plt.legend()
fig = plt.gcf()
fig.savefig(os.path.join(DIR,"accumulated_probability_ps.png"))
plt.show()

In [ ]:
# PLOT SIMULATION
cutoff = 20*60

def plot(alg,d,img):
    fig, ax = plt.subplots()
    
    ax.imshow(img, interpolation=None,origin='bottom', extent=[0, img.shape[0], 0, img.shape[1]], cmap='gray')
    wps = d.pos
    t = np.array(d.t)
    wps_x = np.array(wps.x)
    wps_y = np.array(wps.y)
    ax.plot(wps_x[np.where(t>=cutoff)],wps_y[np.where(t>=cutoff)],'r',linewidth=3)
    ax.plot(wps_x[np.where(t<cutoff)],wps_y[np.where(t<cutoff)],'b',linewidth=3)
        
    ax.set_xlim(-1,img.shape[0]+1)
    ax.set_ylim(-1,img.shape[1]+1)
    ax.set_xlabel("x (m)")
    ax.set_ylabel("y (m)")
    
    s = str(alg).split('.')[1]
    fig.savefig(os.path.join(DIR,s+"_sim.png"))
    ax.set_title(s)
       
    return fig,ax
    
for key in data_sim.data:
    alg, output = key
    fig,ax = plot(alg,output,img)
    plt.tight_layout()
    plt.show()
    break

In [ ]:
# SAR SIM for PS data
import diskcache as dc
import tempfile

objs = np.array(data_sar.data)

for d,label in zip([d1,d2,d3],["PS ASU Inspector","PS AUS Steven Preece","Attempted Parallel Swaths"]):
    wps = data_wp.data[str(alg)]['wps']
    
    print(label)
    vehicle = VehicleSimData()
    cache = dc.Cache(os.path.join(tempfile.gettempdir(),'opp4sar',str(np.random.rand())[2:]))
    
    max_x,max_y = np.max(objs,axis=0)
    min_x,min_y = np.min(objs,axis=0)
    x,y = np.meshgrid(np.arange(min_x,max_x+1),np.arange(min_y,max_y+1))
    x,y=x.flatten(),y.flatten()
    objs_possible_xy = np.vstack((x,y)).T

    for xi,yi,ti in zip(d.utm[0],d.utm[1],d.t_s):
        # create distance vectors
        v = np.array([xi-objs_possible_xy[:,0],
                    yi-objs_possible_xy[:, 1]])

        # if distance is < search radius -> object found
        dist = np.linalg.norm(v, axis=0)

        if any(dist < R):
            inds = np.where(dist < R)[0]
            for ind in inds:
                loc = objs_possible_xy[ind]
                cache[f'{loc[0]},{loc[1]}'] = ti                            
            objs_possible_xy = np.delete(objs_possible_xy, inds, axis=0)

    for key in cache.iterkeys():
        t = cache[key]
        xy=tuple(np.array(key.split(',')).astype(float).astype(int))
        inds = np.where((objs[:,0]==xy[0])&(objs[:,1]==xy[1]))[0]
        for ind in inds:
            vehicle.found.append(
                (t,objs[ind])
            )

    data_sim.data.append((label,vehicle))
    del vehicle, cache

In [ ]:
prob = img
cutoff = 20*60 # 20 mins

total_objects = len(data_sar.data)
colors = np.array([ImageColor.getcolor(f, "RGB") for f in 
         ['#b71c1c',
          '#311b92',
          '#ff6f00', 
          '#4fc3f7',
          '#d500f9',
          '#1b5e20']])/255

dct = {}
for i,data in enumerate(data_sim.data):
    alg = str(data[0])
    if '.' in alg:
        alg = alg.split('.')[1]
    vehicle = data[1]
    print(f"{i} | {alg}: {100*len(vehicle.found)/total_objects:.2f}% found ({len(vehicle.found)} out of {total_objects})")

    # if alg not in (WaypointAlgorithmEnum.PARALLEL_SWATHS, WaypointAlgorithmEnum.PABO_GA):
    #     continue

    t_hist = np.array(vehicle.found)[:,0]
    t_hist = t_hist.astype(float)
    plt.hist(t_hist,density=True,color=colors[i],bins=20)
    
    r = st.gaussian_kde(t_hist)
    mn,mx = (0,cutoff)
    x = np.linspace(mn,mx,1000)
    #l_main = plt.plot(x, r.pdf(x),color=colors[i], label=f"{alg} {100*len(vehicle.found)/total_objects:.2f}%")

    mn,mx = plt.ylim()
    mean = np.mean(t_hist)
    l_mean = plt.plot([mean, mean],[0, 0.003],'-.',color=colors[i],label=f"{alg}")
    print( f"Objects found  = {100*len(vehicle.found)/total_objects:.2f}%")

    dct[alg] = {
        'x':x,
        'y':r.pdf(x),
        'color':colors[i],
        't_hist':t_hist,
        'mean':np.mean(t_hist)
    }
    print(f"{alg} w/ mean time to found = {np.mean(t_hist):.2f}s and found before endurance limit = {100*np.sum(t_hist<cutoff)/len(vehicle.found):.2f}%")


plt.legend()
plt.ylabel("Probability")
plt.xlabel("Time till object found (s)")
fig = plt.gcf()
plt.show()
fig.savefig(os.path.join(DIR,"time_till_object_found_ps.png"),dpi=300)    

In [ ]:
for alg,d in dct.items():
    t_hist = d['t_hist']
    y = []
    x = []
    for i in np.linspace(0,cutoff*1.5):
        x.append(i)
        y.append(100*np.sum(t_hist<i)/total_objects)

    plt.plot(x,y,label=f"{str(alg)}",color=d['color'])

plt.legend(loc='lower right')
plt.ylabel("Percentage found (%)")
plt.xlabel("Endurance limited time-to-find (s)")
fig = plt.gcf()
plt.show()
fig.savefig(os.path.join(DIR,"endurance_limited_time-to-find_ps.png"),dpi=300)
